# Name : **Meghraj Prajapat**
# Roll number : **23M1583**

# This Notebook has two parts
## Part(1) -> Here I have put all the necessary functions which were used to solve problems.
## Part(2) -> In this part I have solved HW4 problems.

# Part 1

In [ ]:
import numpy as np
from sympy import *
import math
import matplotlib.pyplot as pt

In [ ]:
import sys, os


In [ ]:

def block_print():
    sys.stdout = open('/dev/null', 'w')
def enable_print():
    sys.stdout.close()
    sys.stdout = sys.__stdout__

In [ ]:
def four_node_quad():
    z,n=symbols('z n')
    N=np.array([[(1-z)*(1-n)/4,(1+z)*(1+n)/4,(1+z)*(1-n)/4,(1-z)*(1+n)/4]])
    B=(1/4)*np.array([[-(1-n),(1-n),(1+n),-(1+n)],[-(1-z),-(1+z),(1+z),(1-z)]])
    return N,B
z, n = symbols('z n')

In [ ]:
def three_node_tri():
    z,n=symbols('z n')
    N=np.array([[1-z-n,z,n]])
    B=np.array([[-1,1,0],[-1,0,1]])
    return N,B

In [ ]:
def nine_node_quad():
    z,n=symbols('z n')
    N =np.array([[z*n*(1-z)*(1-n)/4,z*n*(1+z)*(n-1)/4,(z*n/4)*(1+z)*(1+n),(z*n/4)*(1+n)*(z-1),(-n/2)*(1-z**2)*(1-n),(z/2)*(1+z)*(1-n**2),
            n/2*(1-z**2)*(1+n),-z/2*(1-z)*(1-n**2),(1-z**2)*(1-n**2)]])

    B=np.array([[n*(n - 1)*(2*z - 1)/4, n*(n - 1)*(2*z + 1)/4,
        n*(n + 1)*(2*z + 1)/4, n*(n + 1)*(2*z - 1)/4, n*z*(1 - n),
        (n**2 - 1)*(-2*z - 1)/2, -n*z*(n + 1), (1 - 2*z)*(n**2 - 1)/2,
        2*z*(n**2 - 1)],
       [z*(2*n - 1)*(z - 1)/4, z*(2*n - 1)*(z + 1)/4,
        z*(2*n + 1)*(z + 1)/4, z*(2*n + 1)*(z - 1)/4,
        (1 - 2*n)*(z**2 - 1)/2, -n*z*(z + 1), (-2*n - 1)*(z**2 - 1)/2,
        n*z*(1 - z), 2*n*(z**2 - 1)]])
    return N,B


In [ ]:
def gauss2D(n):
    x, w = np.polynomial.legendre.leggauss(n)

    weights = []
    gauss_pts = []

    for i in range(n):
        for j in range(n):
            wts = w[i] * w[j]
            weights.append(wts)

            g = [x[i], x[j]]
            gauss_pts.append(g)

    return np.array(weights), np.array(gauss_pts)

In [ ]:
def Formload2D(lpf,g,w,N,detJ1,p,boundterms,x,enodes):#lpf =>[2,]
    z,n=symbols('z n')
    # Define Floc
    ndof=1
    Floc=np.zeros((p*ndof,1))

    # 1)Add Volume term to Floc

    s=lpf[0,1]
    if s!=0:
        if p==4:
            IntF = s*(N.T)*detJ1
            IntF =Matrix(IntF)
            for k in range(len(g[:,0])):
                F1= IntF.subs({z:g[k,0],n:g[k,1]})
                F1 =(np.array(F1)*w[k]).astype(np.float32)
                Floc+=F1
            #Floc+= np.array([[expr.subs({z: g[k,0], n: g[k,1]}) for expr in IntF]]).reshape(p,1).astype(np.float32)

        else:
            print('Traingular source load')
            N =Matrix(N)

            IntF = integrate(integrate(N.T,(z, 0,1-n)), (n, 0,1))
            IntF =np.array(IntF).astype(np.float32)
            print(IntF*s*(detJ1.astype(np.float32)))
            print(detJ1)
            Floc+=(IntF)*s*(detJ1.astype(np.float32))

    #2) Add Boundary integral to Floc

    #
    if p==3:
        for i in boundterms:
            # code for boundary term contribution by lower edge

            if enodes[0]==i[1] and enodes[1]==i[2]:#lower edge of tri
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        q=j[2]
                        print(q)
                        break
                    else:
                        q=0

                N=Matrix(N)
                N=N.subs(n,0)
                Nl= np.array(N)

                #Nl =np.array([expr.subs(n, 0) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5

                #l = ((x[0,0]-x[1,0])**2+(x[0,1]-x[1,1])**2)**0.5
                intF= q*(N.T)*(dl)   # it is a matrix with substituted value

                #F1= np.array([[integrate(expr, (z, 0, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,0,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1




            elif enodes[1]==i[1] and enodes[2]==i[2] :
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        q=j[2]
                        break
                    else:
                        q=0

                # Slant edge of triangle
                N=Matrix(N)
                N=N.subs(n,1-z)
                Nl= np.array(N)

                #Nl =np.array([expr.subs(n, 1-z) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[1,0]-x[2,0])**2+(x[1,1]-x[2,1])**2)**0.5
                intF= q*(N.T)*(dl)
                #intF =np.array([expr.subs(n, 1-z) for expr in intF])
                #F1= np.array([[integrate(expr, (z, 1, 0)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,1,0))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1


            elif enodes[2]==i[1] and enodes[0]==i[2]:
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        q=j[2]
                        break
                    else:
                        q=0
                # Left Egde
                N=Matrix(N)
                N=N.subs(z,0)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(z, 0) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[2,0]-x[0,0])**2+(x[2,1]-x[0,1])**2)**0.5
                intF= q*(N.T)*(dl)
                #intF =np.array([expr.subs(z, 0) for expr in intF])
                #F1= np.array([[integrate(expr, (n, 0, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((n,0,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1

    else:
        #p=4
        # For Quadrilateral
        for i in boundterms:
            if enodes[0]==i[1] and enodes[1]==i[2]:#lower edge of tri
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        q=j[2]
                        print(q)
                        break
                    else:
                        q=0


                # Lower edge
                N=Matrix(N)
                N=N.subs(n,-1)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(n, -1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = (t1**2+t2**2)**0.5
                #l = ((x[0,0]-x[1,0])**2+(x[0,1]-x[1,1])**2)**0.5
                intF= q*(N.T)*(dl)
                #intF =np.array([expr.subs(n, -1) for expr in intF])
                #F1= np.array([[integrate(expr, (z, -1, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,-1,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1



            elif enodes[1]==i[1] and enodes[2]==i[2] :
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        q=j[2]
                        break
                    else:
                        q=0
                print(f"at right edge for nodes")
                print(enodes)
                # right edge
                N=Matrix(N)
                N=N.subs(z,1)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(z, 1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = (t1**2+t2**2)**0.5
                #l = ((x[1,0]-x[2,0])**2+(x[1,1]-x[2,1])**2)**0.5
                intF= q*(N.T)*(dl)
                #intF =np.array([expr.subs(z, 1) for expr in intF])
                #F1= np.array([[integrate(expr, (n, -1, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((n,-1,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1

            elif enodes[2]==i[1] and enodes[3]==i[2]:
                for j in lpf:
                    if j[1]==i[0]and j[0]==2:
                        q=j[2]
                        break
                    else:
                        q=0
                print(q)
                print('it came here')
                # Top egde
                N=Matrix(N)
                N=N.subs(n,1)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(n, 1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[2,0]-x[3,0])**2+(x[2,1]-x[3,1])**2)**0.5
                intF= q*(N.T)*(dl)
                #intF =np.array([expr.subs(n, 1) for expr in intF])
                #F1= np.array([[integrate(expr, (z, 1, -1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,1,-1))
                F1 =np.array(F1).astype(np.float32)
                print(F1)
                Floc+=F1


            elif enodes[3]==i[1] and enodes[0]==i[2]:
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        q=j[2]
                        break
                    else:
                        q=0
                print(q)
                 #left Edge

                N=Matrix(N)
                N=N.subs(z,-1)
                Nl= np.array(N)
                # Nl =np.array([expr.subs(z, -1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[2,0]-x[3,0])**2+(x[2,1]-x[3,1])**2)**0.5
                intF= q*(N.T)*(dl)
                #intF =np.array([expr.subs(z, -1) for expr in intF])
                #F1= np.array([[integrate(expr, (n, 1, -1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((n,1,-1))
                F1 =np.array(F1).astype(np.float32)
                print(F1)
                Floc+=F1







    #...
    return Floc

In [ ]:
def Formload2Dvec(lpf,g,w,N,detJ1,p,boundterms,x,enodes):#lpf =>[2,]
    z,n=symbols('z n')
    # Define Floc
    ndof=2
    Floc=np.zeros((p*ndof,1))

    # 1)Add Volume term to Floc

    sx=lpf[0,1]
    sy=lpf[0,2]
    s=np.array([[sx],[sy]])
    N1 =convert_matrix3(N)
    if sx!=0 or sy!=0:
        if p==4:
            IntF = np.matmul((N1.T),s)*detJ1
            IntF =Matrix(IntF)
            for k in range(len(g[:,0])):
                F1= IntF.subs({z:g[k,0],n:g[k,1]})
                F1 =(np.array(F1)*w[k]).astype(np.float32)
                Floc+=F1
            #Floc+= np.array([[expr.subs({z: g[k,0], n: g[k,1]}) for expr in IntF]]).reshape(p,1).astype(np.float32)

        else:
            print('Traingular source load')
            IntF = np.matmul((N1.T),s)*detJ1
            IntF =Matrix(IntF)

            IntF = integrate(integrate(IntF,(z, 0,1-n)), (n, 0,1))
            IntF =np.array(IntF).astype(np.float32)
            #print(IntF*s*(detJ1.astype(np.float32)))
            #print(detJ1)
            Floc+=(IntF)

    #2) Add Boundary integral to Floc

    #
    if p==3:
        for i in boundterms:
            # code for boundary term contribution by lower edge

            if enodes[0]==i[1] and enodes[1]==i[2]:#lower edge of tri
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        qx=j[2]
                        qy=j[3]
                        break
                    else:
                        qx=0
                        qy=0
                if qx == 0 and qy==0:
                    continue
                print('at lower edge')
                print(enodes)
                q=Matrix([[qx],[qy]])
                N1=Matrix(N1)
                N1=N1.subs(n,0)

                N=Matrix(N)
                N=N.subs(n,0)
                Nl= np.array(N)

                #Nl =np.array([expr.subs(n, 0) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5

                #l = ((x[0,0]-x[1,0])**2+(x[0,1]-x[1,1])**2)**0.5
                intF= (N1.T*q)*(dl)   # it is a matrix with substituted value

                #F1= np.array([[integrate(expr, (z, 0, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,0,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1




            elif enodes[1]==i[1] and enodes[2]==i[2] :
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        qx=j[2]
                        qy=j[3]
                        break
                    else:
                        qx=0
                        qy=0
                if qx == 0 and qy==0:
                    continue


                q=Matrix([[qx],[qy]])
                print(q)
                N1=Matrix(N1)
                N1=N1.subs(n,1-z)

                N=Matrix(N)
                N=N.subs(n,1-z)
                Nl= np.array(N)

                #Nl =np.array([expr.subs(n, 1-z) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[1,0]-x[2,0])**2+(x[1,1]-x[2,1])**2)**0.5
                intF= (N1.T*q)*(dl)
                #intF =np.array([expr.subs(n, 1-z) for expr in intF])
                #F1= np.array([[integrate(expr, (z, 1, 0)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,1,0))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1


            elif enodes[2]==i[1] and enodes[0]==i[2]:
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        qx=j[2]
                        qy=j[3]
                        break
                    else:
                        qx=0
                        qy=0
                if qx == 0 and qy==0:
                    continue
                print('at left edge')
                print(enodes)
                # Left Egde
                q=Matrix([[qx],[qy]])
                N1=Matrix(N1)
                N1=N1.subs(z,0)

                N=Matrix(N)
                N=N.subs(z,0)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(z, 0) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[2,0]-x[0,0])**2+(x[2,1]-x[0,1])**2)**0.5
                intF= (N1.T*q)*(dl)
                #intF =np.array([expr.subs(z, 0) for expr in intF])
                #F1= np.array([[integrate(expr, (n, 0, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((n,0,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1

    else:
        #p=4
        # For Quadrilateral
        for i in boundterms:
            if enodes[3]==i[1] and enodes[0]==i[2]:#lower edge of tri
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        qx=j[2]
                        qy=j[3]
                        break
                    else:
                        qx=0
                        qy=0
                if qx == 0 and qy==0:
                    continue
                print('calculations for lower edge of quad are going')
                print(enodes)
                # Lower edge
                q=Matrix([[qx],[qy]])
                N1=Matrix(N1)
                N1=N1.subs(n,-1)

                N=Matrix(N)
                N=N.subs(n,-1)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(n, -1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[0,0]-x[1,0])**2+(x[0,1]-x[1,1])**2)**0.5
                intF= (N1.T*q)*(dl)
                #intF =np.array([expr.subs(n, -1) for expr in intF])
                #F1= np.array([[integrate(expr, (z, -1, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,-1,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1

             # right edge

            elif enodes[0]==i[1] and enodes[1]==i[2] :
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        qx=j[2]
                        qy=j[3]
                        break
                    else:
                        qx=0
                        qy=0

                if qx == 0 and qy==0:
                    continue


                print('calculations for right edge of quad are going')
                print(enodes)
                q=Matrix([[qx],[qy]])
                N1=Matrix(N1)
                N1=N1.subs(z,1)

                N=Matrix(N)
                N=N.subs(z,1)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(z, 1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = (t1**2+t2**2)**0.5
                #l = ((x[1,0]-x[2,0])**2+(x[1,1]-x[2,1])**2)**0.5
                intF= (N1.T*q)*(dl)
                #intF =np.array([expr.subs(z, 1) for expr in intF])
                #F1= np.array([[integrate(expr, (n, -1, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((n,-1,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1

            # Top egde
            elif enodes[1]==i[1] and enodes[2]==i[2]:
                for j in lpf:
                    if j[1]==i[0]and j[0]==2:
                        qx=j[2]
                        qy=j[3]
                        break
                    else:
                        qx=0
                        qy=0

                print('it came here')

                if qx == 0 and qy==0:
                    continue
                print('calculations for top edge of quad are going')
                print(enodes)
                q=Matrix([[qx],[qy]])
                N1=Matrix(N1)
                N1=N1.subs(n,1)

                N=Matrix(N)
                N=N.subs(n,1)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(n, 1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[2,0]-x[3,0])**2+(x[2,1]-x[3,1])**2)**0.5
                intF= -(N1.T*q)*(dl)
                #intF =np.array([expr.subs(n, 1) for expr in intF])
                #F1= np.array([[integrate(expr, (z, 1, -1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,1,-1))
                F1 =np.array(F1).astype(np.float32)
                print(F1)
                Floc+=F1


                 #left Edge

            elif enodes[2]==i[1] and enodes[3]==i[2]:
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        qx=j[2]
                        qy=j[3]
                        break
                    else:
                        qx=0
                        qy=0
                print(qy)

                if qx == 0 and qy==0:
                    continue
                print('calculations for lower left of quad are going')
                print(enodes)
                q=Matrix([[qx],[qy]])
                N1=Matrix(N1)
                N1=N1.subs(z,-1)

                N=Matrix(N)
                N=N.subs(z,-1)
                Nl= np.array(N)
                # Nl =np.array([expr.subs(z, -1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[2,0]-x[3,0])**2+(x[2,1]-x[3,1])**2)**0.5
                intF= -(N1.T*q)*(dl)
                #intF =np.array([expr.subs(z, -1) for expr in intF])
                #F1= np.array([[integrate(expr, (n, 1, -1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((n,1,-1))
                F1 =np.array(F1).astype(np.float32)
                print(F1)
                Floc+=F1







    #...
    return Floc

In [ ]:
def FEM2Dscalar(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss,shapefun):

    nnodes = nodes[-1, 0].astype(int)

    Kstiff = np.zeros((nnodes, nnodes))
    Fvec = np.zeros((nnodes, 1))

    z, n = symbols('z n')

    for i in range(nel):

        if eleconnect[i, 1] == 4:
            # compute local Stiffness matrix for quad element

            print(f"element{i} is Quad")
            enodes = eleconnect[i, 2:6].astype(int)

            # To get coordinated of nodes
            X1 = []
            for j in enodes:
                x1 = nodes[j - 1, 1:].tolist()
                X1.append(x1)
            X1 = np.array(X1)  # get the coordinates of element
            # Get the interpolating function

            N, B = shapefun()

            J1 = np.matmul(B, X1)

            sympy_matrix = Matrix(J1.tolist())
            detJ1 = sympy_matrix.det()
            J1inv= sympy_matrix.inv()

            J1inv=np.array(J1inv)
            detJ1=np.array([detJ1])

            Int = np.matmul(np.matmul(J1inv, B).T, np.matmul(J1inv, B))*detJ1

            # getting gauss points 2d

            w, g = gauss2D(ngauss)
            Kloc = np.zeros((4, 4))
            for k in range(len(g[:, 0])):
                K0 = np.array([[element.subs({z: g[k, 0], n: g[k, 1]}) for element in row] for row in Int], dtype=float)
                Kloc+=K0*w[k]

            print(Kloc)
            Kstiff[np.ix_(enodes - 1, enodes - 1)] += Kloc

            # Calculating Load vector........

            Floc = Formload2D(lpf,g,w,N,detJ1,eleconnect[i, 1].astype(int),boundterms,X1,enodes)
            print(enodes)

            Fvec[np.ix_(enodes - 1, [0])] += Floc
            #print(Fvec)

        else:
            # computation for traingular element
            print(f"element{i} is Tri")

            enodes = eleconnect[i, 2:5].astype(int)
            # Getting Coordinates
            X1 = []
            for j in enodes:
                x1 = nodes[j - 1, 1:].tolist()
                X1.append(x1)
            X1 = np.array(X1)

            # Calculating Local Stiffness
            N, B = three_node_tri()
            J1 = np.matmul(B, X1)

            sympy_matrix = Matrix(J1.tolist())
            detJ1 = sympy_matrix.det()
            J1inv= sympy_matrix.inv()
            J1inv=np.array(J1inv)
            detJ1=np.array([detJ1])

            Int = np.matmul(np.matmul(J1inv, B).T, np.matmul(J1inv, B))*detJ1

            # getting gauss points 2d
            w, g = gauss2D(ngauss)
            Ival1 = np.zeros((3, 3))

            Ival1 = 0.5 * Int.astype(np.float32)
            print(Ival1)
            Kstiff[np.ix_(enodes - 1, enodes - 1)] += Ival1

            # Calculating Load vector

            Floc = Formload2D(lpf,g,w,N,detJ1,eleconnect[i, 1].astype(int),boundterms,X1,enodes)  # lpf=[[1,s,0,0],[2,q0,elno.,i,j],[2,q0,elno.,i,j]]
            # and give i/p = eleconnect[i,0] => element number
            print(enodes)
            Fvec[np.ix_(enodes - 1, [0])] += Floc
            #print(Fvec)

    dof = np.zeros(nnodes).astype(int)
    ndof = 1  # Scalar field problems
    tdof = nnodes
    count = 0
    for i in range(fdof.shape[0]):  # fixeddofs=[0,1,0]
        for j in range(1, ndof + 1):
            if fdof[i, j] == 1:
                dof[count] = fdof[i, 0] * ndof + j - 1
                count += 1

    dof = dof[:count]
    freedofs = np.setdiff1d(np.arange(0, tdof), dof)

    # Step 4 : Solve KU = F

    U = np.zeros([tdof, 1])

    if len(dof) == 0:
        print('solving a matrix')
        U = np.linalg.solve(Kstiff, np.zeros(Kstiff.shape[0]))
    else:
        U[dof[0]] = fdof[0, -1]
        U[freedofs] = np.linalg.solve(Kstiff[np.ix_(freedofs, freedofs)], Fvec[freedofs] - Kstiff[np.ix_(freedofs, dof)] @ U[dof])
    print(Kstiff)

    F = np.matmul(Kstiff, U)
    return U,F


In [ ]:
def convert_matrix1(Mi):
    # Get the shape of the input matrix
    rows, cols = Mi.shape

    # Create an output matrix with the required shape
    output_matrix = np.zeros((rows * 2, cols * 2),dtype=Mi.dtype)

    for i in range(cols):
        output_matrix[0:2,2*i]=Mi[:,i]
        output_matrix[2:4,2*i+1]=Mi[:,i]

    return output_matrix


In [ ]:
def convert_matrix3(Mi):
    # Get the shape of the input matrix
    rows,cols = Mi.shape

    # Create an output matrix with the required shape
    output_matrix = np.zeros((2, cols * 2),dtype=Mi.dtype)

    for i in range(cols):
        output_matrix[0,2*i]=Mi[0,i]
        output_matrix[1,2*i+1]=Mi[0,i]

    return output_matrix


In [ ]:
def convert_matrix2(Mi):
    # Get the shape of the input matrix
    rows, cols = Mi.shape

    # Create an output matrix with the required shape
    output_matrix = np.zeros((rows * 2, cols * 2),dtype=Mi.dtype)

    output_matrix[0:2,0:2]=Mi
    output_matrix[2:4,2:4]=Mi

    return output_matrix


In [ ]:
def FEM2Dvec(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss,D,shapefun,formload):


    nnodes = nodes[-1, 0].astype(int)
    ndof=2

    Kstiff = np.zeros((nnodes*2, nnodes*2))
    Fvec = np.zeros((nnodes*2, 1))

    z, n = symbols('z n')

    for i in range(nel):
        p=eleconnect[i, 1].astype(int)

        if p == 4 or 9:
            # compute local Stiffness matrix for quad element
            print(f"element{i} is Quad")
            enodes = eleconnect[i, 2:(p+2)].astype(int)

            # To get coordinated of nodes
            X1 = []
            for j in enodes:
                x1 = nodes[j - 1, 1:].tolist()
                X1.append(x1)
            X1 = np.array(X1)  # get the coordinates of element
            # Get the interpolating function

            N, B = shapefun()

            J1 = np.matmul(B, X1)

############################################
           # sympy_matrix = Matrix(J1.tolist())
            #detJ1 = sympy_matrix.det()
            #if i == 124 :
             # J1inv = S1
            #else:
            #J1inv= sympy_matrix.inv()
            #J1inv=np.array(J1inv)
            #print(J1inv)
            #detJ1=np.array([detJ1])
############################################

            #P = convert_matrix2(J1inv)
            #B3 = convert_matrix1(B)
            #B1= np.array([[1,0,0,0],[0,0,0,1],[0,1,1,0]])
            #term=np.matmul(P, B3)
            #term1=np.matmul(B1,term)

            #Int = np.matmul(np.matmul(term1.T,D),term1)*detJ1

            # Size of int => 2n*2n

            # getting gauss points 2d

            w, g = gauss2D(ngauss)
            Kloc = np.zeros((2*p, 2*p))
            for k in range(len(g[:, 0])):
                J1 =np.array([[element.subs({z: g[k, 0], n: g[k, 1]}) for element in row] for row in J1], dtype=float)
                detJ1 = np.linalg.det(J1)
                J1inv = np.linalg.inv(J1)
                P = convert_matrix2(J1inv)
                B3 = convert_matrix1(B)
                B1= np.array([[1,0,0,0],[0,0,0,1],[0,1,1,0]])
                term=np.matmul(P, B3)
                term1=np.matmul(B1,term)

                Int = np.matmul(np.matmul(term1.T,D),term1)*detJ1
                K0 = Int
                Kloc+=K0*w[k]
            print(Kloc)
            enodes=enodes-1
            if p==4:
                e1=np.array([enodes[0]*2, enodes[0]*2+1,enodes[1]*2,enodes[1]*2+1,enodes[2]*2,enodes[2]*2+1,enodes[3]*2,enodes[3]*2+1])
            elif p==9:
                e1=np.array([enodes[0]*2, enodes[0]*2+1,enodes[1]*2,enodes[1]*2+1,enodes[2]*2,enodes[2]*2+1,enodes[3]*2,enodes[3]*2+1,
                            enodes[4]*2,enodes[4]*2+1,enodes[5]*2,enodes[5]*2+1,enodes[6]*2,enodes[6]*2+1,enodes[7]*2,enodes[7]*2+1,
                            enodes[8]*2,enodes[8]*2+1])
            else:
                pass
            Kstiff[np.ix_(e1 , e1)] += Kloc

            # Calculating Load vector........

               # Send converted matrix to form boundary loads

            Floc = formload(lpf,g,w,N,detJ1,p,boundterms,X1,enodes+1)
            print(Floc)


            Fvec[np.ix_(e1, [0])] += Floc
            #print(Fvec)

        else:
            # computation for traingular element
            print(f"element{i} is Tri")

            enodes = eleconnect[i, 2:5].astype(int)
            # Getting Coordinates
            X1 = []
            for j in enodes:
                x1 = nodes[j - 1, 1:].tolist()
                X1.append(x1)
            X1 = np.array(X1)

            # Calculating Local Stiffness
            N, B = three_node_tri()
            J1 = np.matmul(B, X1)

            sympy_matrix = Matrix(J1.tolist())
            detJ1 = sympy_matrix.det()


            J1inv= sympy_matrix.inv()

            J1inv=np.array(J1inv)
            detJ1=np.array([detJ1])

            P = convert_matrix2(J1inv)
            B3 = convert_matrix1(B)
            B1= np.array([[1,0,0,0],[0,0,0,1],[0,1,1,0]])
            term=np.matmul(P, B3)
            term1=np.matmul(B1,term)

            Int = np.matmul(np.matmul(term1.T,D),term1)*detJ1

            # getting gauss points 2d
            w, g = gauss2D(ngauss)
            #Ival1 = np.zeros((6, 6))

            Ival1 = 0.5* Int.astype(np.float32)
            print(Ival1)

            enodes=enodes-1
            e1=np.array([enodes[0]*2, enodes[0]*2+1,enodes[1]*2,enodes[1]*2+1,enodes[2]*2,enodes[2]*2+1])

            Kstiff[np.ix_(e1 , e1)] += Ival1


            # Calculating Load vector

            #N1 = convert_matrix3(N)

            Floc = formload(lpf,g,w,N,detJ1,eleconnect[i, 1].astype(int),boundterms,X1,enodes+1)  # lpf=[[1,s,0,0],[2,q0,elno.,i,j],[2,q0,elno.,i,j]]
            # and give i/p = eleconnect[i,0] => element number
            #print(enodes)
            Fvec[np.ix_(e1, [0])] += Floc
            #print(Fvec)

    # Defining Fvec from given loads
    #for i in lpf:
     #   if i[0]==4:  # load of type 4
      #      Fvec[i[1]-1]=i[2]




    dof = np.zeros(nnodes).astype(int)
    ndof = 2  # vector field problems
    tdof = nnodes*2
    count = 0
    for i in range(fdof.shape[0]):  # fixeddofs=[0,1,0]
        for j in range(1, ndof + 1):
            if fdof[i, j] == 1:
                dof[count] = fdof[i, 0] * ndof + j - 1
                count += 1

    dof = dof[:count]
    freedofs = np.setdiff1d(np.arange(0, tdof), dof)

    # Step 4 : Solve KU = F

    U = np.zeros([tdof, 1])

    if len(dof) == 0:
        print('solving a matrix')
        U = np.linalg.solve(Kstiff, np.zeros(Kstiff.shape[0]))
    else:
        U[dof[0]] = fdof[0, -1]
        U[freedofs] = np.linalg.solve(Kstiff[np.ix_(freedofs, freedofs)], Fvec[freedofs] - Kstiff[np.ix_(freedofs, dof)] @ U[dof])
    print(Kstiff)


    # TO get stresses



    Stress=np.zeros((nel,3),dtype=object)
    for i in range(nel):
        p=eleconnect[i, 1].astype(int)

        if p == 4 or 9:
            # compute local Stiffness matrix for quad element
            print(f"element{i} is Quad")
            enodes = eleconnect[i, 2:(p+2)].astype(int)

            # To get coordinated of nodes
            X1 = []
            for j in enodes:
                x1 = nodes[j - 1, 1:].tolist()
                X1.append(x1)
            X1 = np.array(X1)  # get the coordinates of element
            # Get the interpolating function

            N, B = shapefun()

            J1 = np.matmul(B, X1)

            sympy_matrix = Matrix(J1.tolist())
            detJ1 = sympy_matrix.det()
            if detJ1 == 0:
              raise ValueError("Matrix is singular and cannot be inverted")
            J1inv= sympy_matrix.inv()

            J1inv=np.array(J1inv)
            detJ1=np.array([detJ1])

            P = convert_matrix2(J1inv)
            B3 = convert_matrix1(B)
            B1= np.array([[1,0,0,0],[0,0,0,1],[0,1,1,0]])
            term=np.matmul(P, B3)
            term1=np.matmul(B1,term) # B1PB3


            enodes=enodes-1
            if p==4:
                e1=np.array([enodes[0]*2, enodes[0]*2+1,enodes[1]*2,enodes[1]*2+1,enodes[2]*2,enodes[2]*2+1,enodes[3]*2,enodes[3]*2+1])
            elif p==9:
                e1=np.array([enodes[0]*2, enodes[0]*2+1,enodes[1]*2,enodes[1]*2+1,enodes[2]*2,enodes[2]*2+1,enodes[3]*2,enodes[3]*2+1,
                            enodes[4]*2,enodes[4]*2+1,enodes[5]*2,enodes[5]*2+1,enodes[6]*2,enodes[6]*2+1,enodes[7]*2,enodes[7]*2+1,
                            enodes[8]*2,enodes[8]*2+1])


            U1=U[e1,0] # 1*2n
            print(U1)

            Strains= np.matmul(term1,U1.T)  # 3*2n * 2n*1
            print(Strains)
            St1= np.matmul(D,Strains)   # 3*3 * 3*1
            print(St1)

            Stress[i,:]=St1.T   # 1*3

        else:
            # computation for traingular element
            print(f"element{i} is Tri")

            enodes = eleconnect[i, 2:5].astype(int)
            # Getting Coordinates
            X1 = []
            for j in enodes:
                x1 = nodes[j - 1, 1:].tolist()
                X1.append(x1)
            X1 = np.array(X1)

            # Calculating Local Stiffness
            N, B = three_node_tri()
            J1 = np.matmul(B, X1)

            sympy_matrix = Matrix(J1.tolist())
            detJ1 = sympy_matrix.det()
            J1inv= sympy_matrix.inv()

            J1inv=np.array(J1inv)
            detJ1=np.array([detJ1])

            P = convert_matrix2(J1inv)
            B3 = convert_matrix1(B)
            B1= np.array([[1,0,0,0],[0,0,0,1],[0,1,1,0]])
            term=np.matmul(P, B3)
            term1=np.matmul(B1,term)

            enodes=enodes-1
            e1=np.array([enodes[0]*2, enodes[0]*2+1,enodes[1]*2,enodes[1]*2+1,enodes[2]*2,enodes[2]*2+1])

            U1=U[e1,0] # 1*2n   # Displacements of the element

            Strains= np.matmul(term1,U1.T)  # 3*2n * 2n*1
            St1= np.matmul(D,Strains)   # 3*3 * 3*1

            Stress[i,:]=St1.T



    return U,Stress


In [ ]:
def getdefnodes(nnodes,nodal_data,U62):
  U62x=np.zeros(nnodes)
  U62y= np.zeros(nnodes)
  for i in range(len(U62x)):
    U62x[i]=U62[2*i,0]*4              # *4 is scaling factor
    U62y[i]=U62[2*i+1,0]*4
  U62def=np.zeros((len(U62x),2))
  U62def[:,0]=U62x
  U62def[:,1]=U62y
  nodesdef =np.column_stack((nodal_data[:,0], nodal_data[:,1:3]+U62def[:,0:2]))
  return nodesdef


In [ ]:
def getstress(S):
  Stress=np.zeros((len(S),4))
  for i in range(len(S)):
    s1=S[i].subs(z,-1).subs(n,-1)
    s2=S[i].subs(z,1).subs(n,-1)
    s3=S[i].subs(z,1).subs(n,1)
    s4=S[i].subs(z,-1).subs(n,1)
    Stress[i,:]=np.array([s1,s2,s3,s4])
  return Stress

# Problem 6 Endsem

In [ ]:
eleconnect =np.loadtxt('/content/Rect_Torsion_2X2_elecon.txt')

nel =eleconnect[-1,0].astype(int)

nodes=np.loadtxt('/content/Rect_Torsion_2X2_nodes.txt')
boundterms=np.array([[0,0,0]])
ngauss=2
s=2
q0=0
# Write logic from boundterm, if edge==1 or edge==4 , for that node only define fdof
lpf=np.array([[1,s,0]])

fdof = np.loadtxt('/content/Rect_Torsion_2X2_dbc.txt')

In [ ]:
eleconnect

array([[1., 4., 1., 5., 9., 8., 1.],
       [2., 4., 8., 9., 7., 4., 1.],
       [3., 4., 5., 2., 6., 9., 1.],
       [4., 4., 9., 6., 3., 7., 1.]])

In [ ]:
fdof

array([[2., 1., 0.],
       [3., 1., 0.],
       [6., 1., 0.],
       [3., 1., 0.],
       [4., 1., 0.],
       [7., 1., 0.]])

In [ ]:

U61,F61 = FEM2Dscalar(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss,four_node_quad)

element0 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[1 5 9 8]
element1 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[8 9 7 4]
element2 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[5 2 6 9]
element3 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[9 6 3 7]
[[ 0.66666667  0.          0.          0.         -0.16666667  0.
   0. 

## Last printed part above is GLobal Stiffness matrix

In [ ]:
U61

array([[0.08756112],
       [0.10773227],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.0806846 ],
       [0.        ],
       [0.        ],
       [0.08137225]])

In [ ]:
F61

array([[ 0.03125   ],
       [ 0.03125   ],
       [-0.04057152],
       [-0.02712408],
       [-0.08656785],
       [ 0.0625    ],
       [-0.05401895],
       [-0.0417176 ],
       [ 0.125     ]])

## ii)

In [ ]:
eleconnect =np.loadtxt('/content/Rect_Torsion_4X4_elecon.txt')

nel =eleconnect[-1,0].astype(int)

nodes=np.loadtxt('/content/Rect_Torsion_4X4_nodes.txt')
boundterms=np.array([[0,0,0]])
ngauss=2
s=2
q0=0
# Write logic from boundterm, if edge==1 or edge==4 , for that node only define fdof
lpf=np.array([[1,s,0]])

fdof = np.loadtxt('/content/Rect_Torsion_4X4_dbc.txt')

In [ ]:

U62,F62 = FEM2Dscalar(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss,four_node_quad)

element0 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[ 1  5 20 16]
element1 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[ 5  6 18 20]
element2 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[20 18 17 19]
element3 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[16 20 19 15]
element4 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -

In [ ]:
F62

array([[ 0.0078125 ],
       [ 0.0078125 ],
       [-0.01147311],
       [-0.01443423],
       [-0.09651466],
       [ 0.015625  ],
       [ 0.015625  ],
       [ 0.015625  ],
       [-0.07413252],
       [-0.03210991],
       [-0.02704243],
       [-0.04147666],
       [-0.03000355],
       [-0.04781293],
       [ 0.015625  ],
       [ 0.015625  ],
       [ 0.03125   ],
       [ 0.03125   ],
       [ 0.03125   ],
       [ 0.03125   ],
       [ 0.03125   ],
       [ 0.03125   ],
       [ 0.03125   ],
       [ 0.03125   ],
       [ 0.03125   ]])

In [ ]:
U62

array([[0.06434727],
       [0.09882438],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.08288083],
       [0.09628682],
       [0.08456717],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.06568241],
       [0.06320929],
       [0.07298627],
       [0.07906824],
       [0.06729491],
       [0.0736524 ],
       [0.04670795],
       [0.04330269],
       [0.0619104 ],
       [0.08378426],
       [0.03441933]])

## iii)

In [ ]:
eleconnect =np.loadtxt('/content/Rect_Torsion_8X8_elecon.txt')

nel =eleconnect[-1,0].astype(int)

nodes=np.loadtxt('/content/Rect_Torsion_8X8_nodes.txt')
boundterms=np.array([[0,0,0]])
ngauss=2
s=2
q0=0
# Write logic from boundterm, if edge==1 or edge==4 , for that node only define fdof
lpf=np.array([[1,s,0]])

fdof = np.loadtxt('/content/Rect_Torsion_8X8_dbc.txt')

In [ ]:

U63,F63 = FEM2Dscalar(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss,four_node_quad)

element0 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[ 1  5 44 32]
element1 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[ 5  6 42 44]
element2 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[44 42 36 43]
element3 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -0.16666667]
 [-0.16666667  0.66666667 -0.16666667 -0.33333333]
 [-0.33333333 -0.16666667  0.66666667 -0.16666667]
 [-0.16666667 -0.33333333 -0.16666667  0.66666667]]
[32 44 43 31]
element4 is Quad
[[ 0.66666667 -0.16666667 -0.33333333 -

In [ ]:
U63

array([[0.04571393],
       [0.05976199],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.06502766],
       [0.0795524 ],
       [0.08380345],
       [0.0805911 ],
       [0.0722957 ],
       [0.06378244],
       [0.05142737],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.04472429],
       [0.06496183],
       [0.07644174],
       [0.07929222],
       [0.07279024],
       [0.05129927],
       [0.07136744],
       [0.08177474],
       [0.07665967],
       [0.07880035],
       [0.04626783],
       [0.04906078],
       [0.04939455],
       [0.06360988],
       [0.0321461 ],
       [0.0679745 ],
       [0.0743074 ],
       [0.05991886],
       [0.08337847],
       [0.08202953],
       [0.08072233],
       [0.078

In [ ]:
F63

array([[ 0.00195312],
       [ 0.00195312],
       [-0.00389353],
       [-0.0079195 ],
       [-0.07818781],
       [ 0.00390625],
       [ 0.00390625],
       [ 0.00390625],
       [ 0.00390625],
       [ 0.00390625],
       [ 0.00390625],
       [ 0.00390625],
       [-0.05308796],
       [-0.0353585 ],
       [-0.0287037 ],
       [-0.02397244],
       [-0.01833764],
       [-0.01023657],
       [-0.0101709 ],
       [-0.01799983],
       [-0.02288437],
       [-0.02582945],
       [-0.02710077],
       [-0.0262422 ],
       [-0.01701976],
       [-0.03055507],
       [ 0.00390625],
       [ 0.00390625],
       [ 0.00390625],
       [ 0.00390625],
       [ 0.00390625],
       [ 0.00390625],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0.0078125 ],
       [ 0

## iv)

In [ ]:
eleconnect =np.loadtxt('/content/Rect_Torsion_4X4_9node_elecon.txt')

nel =eleconnect[-1,0].astype(int)

nodes=np.loadtxt('/content/Rect_Torsion_4X4_9node_nodes.txt')
boundterms=np.array([[0,0,0]])
ngauss=2
s=2
q0=0
# Write logic from boundterm, if edge==1 or edge==4 , for that node only define fdof
lpf=np.array([[1,s,0]])

fdof = np.loadtxt('/content/Rect_Torsion_4X4_9node_dbc.txt')

In [ ]:

U64,F64 = FEM2Dscalar(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss,nine_node_quad)

element0 is Tri
[[ 5.000000e-01 -5.000000e-01  7.696066e-13]
 [-5.000000e-01  1.000000e+00 -5.000000e-01]
 [ 7.696066e-13 -5.000000e-01  5.000000e-01]]
Traingular source load
[[0.00520833]
 [0.00520833]
 [0.00520833]]
[0.0156250000000017]


ValueError: operands could not be broadcast together with shapes (9,1) (3,1) (9,1) 

# Part 2: Problem Solving

## Data for problem 3

In [ ]:
# For problem 3
nel=3
eleconnect=np.array([[1,4,1,4,5,6,1],[2,3,4,2,5,0,1],[3,3,6,5,3,0,1]])
nodes=np.array([[1,0,0],[2,1,0],[3,0,1],[4,0.6,0],[5,0.6,0.64],[6,0,0.64]])
boundterms=np.array([[1,1,4],[1,4,2],[2,2,5],[2,5,3],[3,3,6],[3,6,1]])
s=2
q0=0
lpf=np.array([[1,s,0]])
ngauss=2
fdof=np.array([[2,1,0],[3,1,0],[6,1,0],[3,1,0]])   # u=0 at node 2 , 5 , 3

In [ ]:
#blockPrint()
U3,F3 = FEM2Dscalar(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss)

element0 is Quad
[[ 0.66805556 -0.19930556 -0.33402778 -0.13472222]
 [-0.19930556  0.66805556 -0.13472222 -0.33402778]
 [-0.33402778 -0.13472222  0.66805556 -0.19930556]
 [-0.13472222 -0.33402778 -0.19930556  0.66805556]]
0
[[0.]
 [0.]
 [0.]
 [0.]]
[1 4 5 6]
[[0.192]
 [0.   ]
 [0.   ]
 [0.192]
 [0.192]
 [0.192]]
element1 is Tri
[[ 1.1125 -0.8    -0.3125]
 [-0.8     0.8     0.    ]
 [-0.3125  0.      0.3125]]
Traingular source load
[[0.08533334]
 [0.08533334]
 [0.08533334]]
[0.256000000000000]
[4 2 5]
[[0.192     ]
 [0.08533334]
 [0.        ]
 [0.27733334]
 [0.27733334]
 [0.192     ]]
element2 is Tri
[[ 1.1333333 -0.3       -0.8333333]
 [-0.3        0.3        0.       ]
 [-0.8333333  0.         0.8333333]]
Traingular source load
[[0.072]
 [0.072]
 [0.072]]
[0.216000000000000]
[6 5 3]
[[0.192     ]
 [0.08533334]
 [0.072     ]
 [0.27733334]
 [0.34933335]
 [0.26400001]]
[[ 0.66805556  0.          0.         -0.19930556 -0.33402778 -0.13472222]
 [ 0.          0.80000001  0.         -0.8000

In [ ]:
e1=np.array([0,1,3,5])

In [ ]:
U3

array([[0.40452026],
       [0.        ],
       [0.        ],
       [0.24264555],
       [0.        ],
       [0.2218001 ]])

In [ ]:
U3[e1,0]

array([0.40452026, 0.        , 0.24264555, 0.2218001 ])

In [ ]:
F3

array([[0.192     ],
       [0.08533334],
       [0.072     ],
       [0.27733334],
       [0.34933335],
       [0.26400001]])

## Data for problem 4

In [ ]:
# Problem 4 Data
nel=2

eleconnect = np.array([[1,3,1,2,5,0,1],[2,4,5,2,3,4,1]])
                      # el no. num nodes,nodes
nodes=np.array([[1,0,0],[2,2,0.5],[3,2,1],[4,0,1],[5,0,0.5]])

boundterms=np.array([[1,1,2],[2,2,3],[3,3,4],[4,4,5],[4,5,1]])
s=9
q0=-15
lpf=np.array([[1,s,0],[2,3,q0]])#,[2,4,0],[2,1,0],[2,2,0]])
              #type     #type2, boundary no. ,value
ngauss=2

fdof=np.array([[0,1,0],[1,1,0],[4,1,0],[3,1,0]]) # u=0 at node 1 , 2 , 5,4

In [ ]:

U4, F4 =FEM2Dscalar(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss)

element0 is Tri
[[ 2.     0.    -2.   ]
 [ 0.     0.125 -0.125]
 [-2.    -0.125  2.125]]
Traingular source load
[[1.5]
 [1.5]
 [1.5]]
[1.00000000000000]
[1 2 5]
[[1.5]
 [1.5]
 [0. ]
 [0. ]
 [1.5]]
element1 is Quad
[[ 1.41666667  0.58333333 -0.70833333 -1.29166667]
 [ 0.58333333  1.41666667 -1.29166667 -0.70833333]
 [-0.70833333 -1.29166667  1.41666667  0.58333333]
 [-1.29166667 -0.70833333  0.58333333  1.41666667]]
at right edge for nodes
[5 2 3 4]
-15
it came here
[[0.]
 [0.]
 [0.]
 [0.]]
0
[[0.]
 [0.]
 [0.]
 [0.]]
[5 2 3 4]
[[1.5       ]
 [3.75000003]
 [2.25000003]
 [2.25000003]
 [3.75000003]]
[[ 2.          0.          0.          0.         -2.        ]
 [ 0.          1.54166667 -1.29166667 -0.70833333  0.45833333]
 [ 0.         -1.29166667  1.41666667  0.58333333 -0.70833333]
 [ 0.         -0.70833333  0.58333333  1.41666667 -1.29166667]
 [-2.          0.45833333 -0.70833333 -1.29166667  3.54166667]]


array([[ 0.        ],
       [-2.05147062],
       [ 2.25000003],
       [ 0.9264706 ],
       [-1.12500001]])

In [ ]:
U4

array([[0.        ],
       [0.        ],
       [1.58823532],
       [0.        ],
       [0.        ]])

## Scalar field problem

## Problem 4 Part 2

In [ ]:
eleconnect =np.loadtxt('Pr1_4q_elecon.txt')

nel =eleconnect[-1,0].astype(int)

nodes=np.loadtxt('Pr1_4q_nodes.txt')

boundterms=np.array([[1,1,5],[1,5,6],[1,6,7],[1,7,8],[1,8,9],[1,9,10],[1,10,11],[1,11,12],[1,12,13],[1,13,14],[1,14,2],
                    [2,2,15],[2,15,16],[2,16,17],[2,17,3],[3,3,18],[3,18,19],[3,19,20],[3,20,21],[3,21,22],[3,22,23],[3,23,24],
                    [3,24,25],[3,25,26],[3,26,27],[3,27,4],[4,4,28],[4,28,29],[4,29,30],[4,30,1]])
ngauss=2
s=9
q0=-15
# Write logic from boundterm, if edge==1 or edge==4 , for that node only define fdof
lpf=np.array([[1,s,0],[2,3,q0]])
f = np.zeros((len(boundterms), 3))
for i in range(len(boundterms[:,0])):
    if boundterms[i,0] == 1 or boundterms[i, 0] == 4:
        f[i,0] = boundterms[i,1]-1           # indices starts from 0
        f[i,1]=1
        f[i,2]=0
    else:
        pass
fdof = f[~np.all(f == 0, axis=1)]

In [ ]:

U42,F42 = FEM2Dscalar(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss)

In [ ]:
U42

## Problem 5.

In [ ]:
eleconnect =np.loadtxt('Pr5_4q_elecon.txt')
nodes = np.loadtxt('Pr5_4q_nodes.txt')
nnodes = nodes[-1, 0].astype(int)
nel = eleconnect[-1,0].astype(int)

boundterms=np.array([[1,1,5],[1,5,6],[1,6,7],[1,7,8],[1,8,9],[1,9,10],[1,10,11],[1,11,2],[2,2,12],[2,12,3],[3,3,13],
                    [3,13,14],[3,14,15],[3,15,16],[3,16,17],[3,17,18],[3,18,19],[3,19,4],[4,4,20],[4,20,1]])

E=200e5
v=0.3
D=np.array([[1,v,0],[v,1,0],[0,0,(1-v)/2]])*(E/(1-v**2))
ngauss=3
sx=0   # body load
sy=0
qx=0  # External traction
qy=100 # stress

# Write logic from boundterm, if edge==1 or edge==4 , for that node only define fdof
fdof=np.array([[0,1,1,0,0],[19,1,1,0,0],[3,1,1,0,0]])  # U=0 at nodes 1, 20 , 4
lpf=np.array([[1,sx,sy,0],[2,2,qx,qy]])

             # type, boundar no. , qx, qy


In [ ]:
U5, F5 =FEM2Dvec(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss,D,four_node_quad,Formload2Dvec)

element0 is Quad
[[  9340659.34064804   3571428.57143689    274725.27470696
    -274725.2747199   -4670329.67033149  -3571428.57143273
   -4945054.94502351    274725.27471574]
 [  3571428.57143689  19340659.34059137    274725.27473065
    8131868.13183741  -3571428.57143273  -9670329.67030327
    -274725.27473481 -17802197.80212551]
 [   274725.27470696    274725.27473065   9340659.34065236
   -3571428.57142025  -4945054.94504141   -274725.27473481
   -4670329.67031791   3571428.57142441]
 [  -274725.2747199    8131868.13183741  -3571428.57142025
   19340659.34063188    274725.27471574 -17802197.80216258
    3571428.57142441  -9670329.67030671]
 [ -4670329.67033149  -3571428.57143273  -4945054.94504141
     274725.27471574   9340659.34066436   3571428.57143689
     274725.27470855   -274725.2747199 ]
 [ -3571428.57143273  -9670329.67030326   -274725.27473481
  -17802197.80216258   3571428.57143689  19340659.34062515
     274725.27473065   8131868.1318407 ]
 [ -4945054.94502351   -27472

In [ ]:
U5

array([[ 0.00000000e+00],
       [ 0.00000000e+00],
       [-2.03002072e-03],
       [-2.79452948e-02],
       [ 2.02904356e-03],
       [-2.79405084e-02],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [-4.95686424e-04],
       [-6.78260090e-04],
       [-9.30376134e-04],
       [-2.49435818e-03],
       [-1.29026954e-03],
       [-5.30983660e-03],
       [-1.57872850e-03],
       [-8.93426049e-03],
       [-1.79498997e-03],
       [-1.31898808e-02],
       [-1.93908914e-03],
       [-1.78962307e-02],
       [-2.01098092e-03],
       [-2.28710887e-02],
       [ 4.88554988e-07],
       [-2.79424071e-02],
       [ 2.01154385e-03],
       [-2.28739601e-02],
       [ 1.93916684e-03],
       [-1.78956756e-02],
       [ 1.79493584e-03],
       [-1.31899659e-02],
       [ 1.57874562e-03],
       [-8.93425243e-03],
       [ 1.29026538e-03],
       [-5.30983593e-03],
       [ 9.30376931e-04],
       [-2.49435878e-03],
       [ 4.95686270e-04],
       [-6.78259899e-04],
       [ 0.0

## Getting Deformed coordinates

In [ ]:
U5x=np.zeros(nnodes)
U5y= np.zeros(nnodes)
for i in range(len(U5x)):
    U5x[i]=U5[2*i,0]*5
    U5y[i]=U5[2*i+1,0]*5

In [ ]:
U5def=np.zeros((len(U5x),2))
U5def[:,0]=U5x
U5def[:,1]=U5y
nodesdef =np.column_stack((nodes[:,0], nodes[:,1:3]+U5def[:,0:2]))
nodesdef

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.00000000e+00,  9.98984990e+00, -1.39726474e-01],
       [ 3.00000000e+00,  1.00101452e+01,  8.60297458e-01],
       [ 4.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 5.00000000e+00,  1.24752157e+00, -3.39130045e-03],
       [ 6.00000000e+00,  2.49534812e+00, -1.24717909e-02],
       [ 7.00000000e+00,  3.74354865e+00, -2.65491830e-02],
       [ 8.00000000e+00,  4.99210636e+00, -4.46713025e-02],
       [ 9.00000000e+00,  6.24102505e+00, -6.59494039e-02],
       [ 1.00000000e+01,  7.49030455e+00, -8.94811534e-02],
       [ 1.10000000e+01,  8.73994510e+00, -1.14355444e-01],
       [ 1.20000000e+01,  1.00000024e+01,  3.60287965e-01],
       [ 1.30000000e+01,  8.76005772e+00,  8.85630200e-01],
       [ 1.40000000e+01,  7.50969583e+00,  9.10521622e-01],
       [ 1.50000000e+01,  6.25897468e+00,  9.34050170e-01],
       [ 1.60000000e+01,  5.00789373e+00,  9.55328738e-01],
       [ 1.70000000e+01,  3.75645133e+00

In [ ]:
np.savetxt("U5defnodes.txt", nodesdef)

In [ ]:
F5.shape  # 16 elements , for each Sx , Sy , Sxy

(16, 3)

In [ ]:
z,n=np.symbols('z n')

In [ ]:
def getstress(S):
  Stress=np.zeros((len(S),4))
  for i in range(len(S)):
    s1=S[i].subs(z,-1).subs(n,-1)
    s2=S[i].subs(z,1).subs(n,-1)
    s3=S[i].subs(z,1).subs(n,1)
    s4=S[i].subs(z,-1).subs(n,1)
    Stress[i,:]=np.array([s1,s2,s3,s4])
  return Stress


In [ ]:
Sx=F5[:,0] # 0th column is Sx


-8715.36570381985

In [ ]:
TensileStress=getstress(Sx)

In [ ]:
np.min(TensileStress)

-8715.365703819853

## Problem 5, Solving Using 9-node Quad

In [ ]:
eleconnect =np.loadtxt('Pr5_9q_elecon.txt')
nodes = np.loadtxt('Pr5_9q_nodes.txt')
nnodes = nodes[-1, 0].astype(int)
nel = eleconnect[-1,0].astype(int)

boundterms=np.array([[1,1,5],[1,5,6],[1,6,7],[1,7,2],[2,2,3],[2,12,3],[3,3,13],
                    [3,13,14],[3,14,15],[3,15,4],[4,4,1]])

E=200e5
v=0.3
D=np.array([[1,v,0],[v,1,0],[0,0,(1-v)/2]])*(E/(1-v**2))
ngauss=4
sx=0   # body load
sy=0
qx=0  # External traction
qy=1000 # stress

# Write logic from boundterm, if edge==1 or edge==4 , for that node only define fdof
fdof=np.array([[0,1,1,0,0],[19,1,1,0,0],[3,1,1,0,0]])  # U=0 at nodes 1, 20 , 4
lpf=np.array([[1,sx,sy,0],[2,2,qx,qy]])

             # type, boundar no. , qx, qy


In [ ]:
eleconnect

array([[ 1.,  9.,  1.,  5., 15.,  4.,  8., 21., 19., 20., 22.,  1.],
       [ 2.,  9.,  5.,  6., 14., 15.,  9., 23., 18., 21., 24.,  1.],
       [ 3.,  9.,  6.,  7., 13., 14., 10., 25., 17., 23., 26.,  1.],
       [ 4.,  9.,  7.,  2.,  3., 13., 11., 12., 16., 25., 27.,  1.]])

In [ ]:
p=9
enodes = eleconnect[3, 2:(p+2)].astype(int)
enodes

array([ 7,  2,  3, 13, 11, 12, 16, 25, 27])

In [ ]:
U52, F52 =FEM2Dvec(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss,D,nine_node_quad,Formload2Dvec)

element0 is Quad
[[ 8.71794872e+06  3.57142857e+06 -1.10500610e+06  9.15750916e+04
  -3.11355311e+05 -3.96825397e+05  1.70940171e+05 -9.15750916e+04
  -1.34310134e+05 -3.66300366e+05  1.90476190e+06  1.58730159e+06
   1.20879121e+06  1.58730159e+06 -5.47008547e+06  3.66300366e+05
  -4.98168498e+06 -6.34920635e+06]
 [ 3.57142857e+06  1.80512821e+07 -9.15750916e+04 -4.13675214e+06
  -3.96825397e+05 -6.44688645e+05  9.15750916e+04  2.20268620e+06
   3.66300366e+05  7.45299145e+06  1.58730159e+06  4.95238095e+06
   1.58730159e+06  1.49450549e+06 -3.66300366e+05 -1.90573871e+07
  -6.34920635e+06 -1.03150183e+07]
 [-1.10500610e+06 -9.15750916e+04  8.71794872e+06 -3.57142857e+06
   1.70940171e+05  9.15750916e+04 -3.11355311e+05  3.96825397e+05
  -1.34310134e+05  3.66300366e+05 -5.47008547e+06 -3.66300366e+05
   1.20879121e+06 -1.58730159e+06  1.90476190e+06 -1.58730159e+06
  -4.98168498e+06  6.34920635e+06]
 [ 9.15750916e+04 -4.13675214e+06 -3.57142857e+06  1.80512821e+07
  -9.15750916e+04  2

In [ ]:
U52

array([[ 0.00000000e+00],
       [ 0.00000000e+00],
       [-2.52394472e-02],
       [-3.52119646e-01],
       [ 2.52657640e-02],
       [-3.52136725e-01],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [-1.19711122e-02],
       [-3.15863248e-02],
       [-2.01525939e-02],
       [-1.13616408e-01],
       [-2.44455327e-02],
       [-2.26845883e-01],
       [-6.23188760e-03],
       [-8.50838831e-03],
       [-1.63874658e-02],
       [-6.75176687e-02],
       [-2.26371725e-02],
       [-1.67550646e-01],
       [-2.50438051e-02],
       [-2.89060692e-01],
       [ 1.68538777e-05],
       [-3.52117018e-01],
       [ 2.44475630e-02],
       [-2.26842156e-01],
       [ 2.01530876e-02],
       [-1.13616235e-01],
       [ 1.19711811e-02],
       [-3.15863497e-02],
       [ 2.50568319e-02],
       [-2.89089527e-01],
       [ 2.26367795e-02],
       [-1.67550763e-01],
       [ 1.63873995e-02],
       [-6.75176369e-02],
       [ 6.23187795e-03],
       [-8.50837977e-03],
       [ 0.0

In [ ]:
defnodes=getdefnodes(nnodes,nodes,U52)

In [ ]:
nodes

array([[ 1.  ,  0.  ,  0.  ],
       [ 2.  , 10.  ,  0.  ],
       [ 3.  , 10.  ,  1.  ],
       [ 4.  ,  0.  ,  1.  ],
       [ 5.  ,  2.5 ,  0.  ],
       [ 6.  ,  5.  ,  0.  ],
       [ 7.  ,  7.5 ,  0.  ],
       [ 8.  ,  1.25,  0.  ],
       [ 9.  ,  3.75,  0.  ],
       [10.  ,  6.25,  0.  ],
       [11.  ,  8.75,  0.  ],
       [12.  , 10.  ,  0.5 ],
       [13.  ,  7.5 ,  1.  ],
       [14.  ,  5.  ,  1.  ],
       [15.  ,  2.5 ,  1.  ],
       [16.  ,  8.75,  1.  ],
       [17.  ,  6.25,  1.  ],
       [18.  ,  3.75,  1.  ],
       [19.  ,  1.25,  1.  ],
       [20.  ,  0.  ,  0.5 ],
       [21.  ,  2.5 ,  0.5 ],
       [22.  ,  1.25,  0.5 ],
       [23.  ,  5.  ,  0.5 ],
       [24.  ,  3.75,  0.5 ],
       [25.  ,  7.5 ,  0.5 ],
       [26.  ,  6.25,  0.5 ],
       [27.  ,  8.75,  0.5 ]])

In [ ]:
np.savetxt("U5defnodes1.txt", defnodes)

## Use this problem as Reference for input

## Problem 6 , Part (ii)

In [ ]:
eleconnect =np.loadtxt('Pr6_9Q1_elecon.txt').astype(np.int64)
nodes = np.loadtxt('Pr6_9Q1_nodes.txt',converters=float)
nnodes = nodes[-1, 0].astype(int)
nel = eleconnect[-1,0].astype(int)

boundterms=np.array([[2,2,25],[2,25,31],[2,31,26],[2,26,32],[2,32,27],[2,27,33],[2,33,28],
                    [2,28,34],[2,34,29],[2,29,35],[2,35,30],[2,30,3]])

E=200e5
v=0.3
D=np.array([[1,v,0],[v,1,0],[0,0,(1-v)/2]])*(E/(1-v**2))
ngauss=3
sx=0   # body load
sy=0
qx=-5000  # External traction
qy=0

# Write logic from boundterm, if edge==1 or edge==4 , for that node only define fdof
fdof=np.array([[0,0,1,0,0],[5,0,1,0,0],[15,0,1,0,0],[16,0,1,0,0],[13,0,1,0,0],[17,0,1,0,0],[12,0,1,0,0]
               ,[18,0,1,0,0],[11,0,1,0,0],[19,0,1,0,0],[10,0,1,0,0],[20,0,1,0,0],[9,0,1,0,0],[21,0,1,0,0],
              [8,0,1,0,0],[22,0,1,0,0],[7,0,1,0,0],[23,0,1,0,0],[6,0,1,0,0],[1,0,1,0,0],[4,1,0,0,0],[60,1,0,0,0],
              [64,1,0,0,0],[59,1,0,0,0],[63,1,0,0,0],[58,1,0,0,0],[62,1,0,0,0],[57,1,0,0,0],[61,1,0,0,0]
              ,[56,1,0,0,0],[3,1,0,0,0]])  # Uy=0 at nodes 1, 6, 16,17,14,18,13,19,12,20,11,21,10,22,9,23,8,24,7,2
lpf=np.array([[1,sx,sy,0],[2,2,qx,qy]])                            # Ux=0 at 5,61,65,60,64,59,63,58,62,57,4

             # type, boundar no. , qx, qy


NameError: name 'np' is not defined

In [ ]:
U62, F62 =FEM2Dvec(nodes, eleconnect,boundterms, fdof, nel, lpf, ngauss,D,four_node_quad,Formload2Dvec)

Streaming output truncated to the last 5000 lines.
 [ 3544200.9017977  -5030611.76391306  -322122.49379695  1572552.07645103
  -3516664.07463184 10236845.96421801   294585.66663109 -6778786.27675598]
 [-5316551.05166514  -321675.75323151 -4819879.66807958 -3598810.43407321
    584701.03682095   294585.66663109  9551729.68292378  3625900.52067363]
 [  227774.79621904  1657014.82143077 -3598810.43407321 -5286704.15581293
   -254864.88281946 -6778786.27675598  3625900.52067363 10408475.61113814]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
element18 is Quad
[[(0.00138502675364194*z - 0.00138502675364105)/(0.00154002110846074*n + 0.00903902794267998*z - 1.15434773951312)
  (1.03003991178391 - 0.00138502675364194*n)/(0.00154002110846074*n + 0.00903902794267998*z - 1.15434773951312)]
 [(0.00876698766833783*z - 1.12067410744868)/(0.00154002110846074*n + 0.00903902794267998*z - 1.15434773951312)
  (-0.00876698766833783*n - 0.00626755992391637)/(0.00154002110846074*n + 0.00903902794267998*

KeyboardInterrupt: 

In [ ]:
M=Matrix([[0.0234368744480495*n + 1.067650629138, 0.0334969065057957*n - 0.084813565107241], [0.0234368744480495*z + 0.0115552381522346, 0.0334969065057957*z + 0.95681917150366]])
S = M.inv()
# Similar problem with element 189

In [ ]:
print(S)

Matrix([[(0.0334969065057957*z + 0.95681917150366)/(0.0220377860599804*n + 0.0377507581819994*z + 1.02252863137054), (0.0848135651072411 - 0.0334969065057957*n)/(0.0220377860599804*n + 0.0377507581819994*z + 1.02252863137054)], [(-0.0234368744480495*z - 0.0115552381522346)/(0.0220377860599804*n + 0.0377507581819994*z + 1.02252863137054), (0.0234368744480495*n + 1.067650629138)/(0.0220377860599804*n + 0.0377507581819994*z + 1.02252863137054)]])


In [ ]:
U62=U62*10

In [ ]:
U62x=np.zeros(nnodes)
U62y= np.zeros(nnodes)
for i in range(len(U62x)):
    U62x[i]=U62[2*i,0]
    U62y[i]=U62[2*i+1,0]

In [ ]:
U62def=np.zeros((len(U62x),2))
U62def[:,0]=U62x
U62def[:,1]=U62y
nodesdef =np.column_stack((nodes[:,0], nodes[:,1:3]+U62def[:,0:2]))
nodesdef

array([[ 1.00000000e+00,  2.16547119e+01,  0.00000000e+00],
       [ 2.00000000e+00,  1.03072262e+02,  0.00000000e+00],
       [ 3.00000000e+00,  9.88609571e+01,  2.35973761e+01],
       [ 4.00000000e+00,  0.00000000e+00,  1.39833453e+01],
       [ 5.00000000e+00,  0.00000000e+00, -7.35147509e-01],
       [ 6.00000000e+00,  2.42156368e+01,  0.00000000e+00],
       [ 7.00000000e+00,  1.00403152e+02,  0.00000000e+00],
       [ 8.00000000e+00,  9.32445894e+01,  0.00000000e+00],
       [ 9.00000000e+00,  8.41417648e+01,  0.00000000e+00],
       [ 1.00000000e+01,  7.45753556e+01,  0.00000000e+00],
       [ 1.10000000e+01,  6.51762342e+01,  0.00000000e+00],
       [ 1.20000000e+01,  5.59639485e+01,  0.00000000e+00],
       [ 1.30000000e+01,  4.68011463e+01,  0.00000000e+00],
       [ 1.40000000e+01,  3.79517572e+01,  0.00000000e+00],
       [ 1.50000000e+01,  3.01067042e+01,  1.52969984e-01],
       [ 1.60000000e+01,  2.68190313e+01,  0.00000000e+00],
       [ 1.70000000e+01,  3.37856407e+01

In [ ]:
np.savetxt('defnodes.txt', nodesdef)

In [ ]:
a = np.loadtxt("/content/defnodes.txt")

FileNotFoundError: /content/defnodes.txt not found.

## Problem 6 , part(i)

In [ ]:
eleconnect =np.loadtxt('P61eleconnect.txt')
nodes = np.loadtxt('P61nodes.txt')
boundterms= np.loadtxt('P61boundary.txt')

In [ ]:
nnodes = nodes[-1, 0].astype(int)
nel = eleconnect[-1,0].astype(int)


ngauss=2
s=0
q0=0

lpf=np.array([[1,s,0]])
f = np.zeros((len(boundterms), 3))
for i in range(len(boundterms[:,0])):
    if boundterms[i,0] == 1 or boundterms[i, 0] == 5:
        f[i,0] = boundterms[i,1]-1           # indices starts from 0
        f[i,1]=1
        f[i,2]=0
    elif boundterms[i,0]==3:
        f[i,0]=boundterms[i,1]-1
        f[i,1]=1
        f[i,2]=10             # => B*U0 =2*5 (let)
    elif boundterms[i,0]==2:
        f[i,0]=boundterms[i,1]-1
        f[i,1]=1
        f[i,2]= nodes[boundterms[i,1]-1,1]*5
fdof = f[~np.all(f == 0, axis=1)]


In [ ]:
def Formload2DPressure(lpf,g,w,N,detJ1,p,boundterms,x,enodes):#lpf =>[2,]
    z,n=symbols('z n')
    # Define Floc
    ndof=2
    Floc=np.zeros((p*ndof,1))

    # 1)Add Volume term to Floc

    sx=lpf[0,1]
    sy=lpf[0,2]
    s=np.array([[sx],[sy]])
    N1 =convert_matrix3(N)
    if sx!=0 or sy!=0:
        if p==4:
            IntF = np.matmul((N1.T),s)*detJ1
            IntF =Matrix(IntF)
            for k in range(len(g[:,0])):
                F1= IntF.subs({z:g[k,0],n:g[k,1]})
                F1 =(np.array(F1)*w[k]).astype(np.float32)
                Floc+=F1
            #Floc+= np.array([[expr.subs({z: g[k,0], n: g[k,1]}) for expr in IntF]]).reshape(p,1).astype(np.float32)

        else:
            print('Traingular source load')
            IntF = np.matmul((N1.T),s)*detJ1
            IntF =Matrix(IntF)

            IntF = integrate(integrate(IntF,(z, 0,1-n)), (n, 0,1))
            IntF =np.array(IntF).astype(np.float32)
            #print(IntF*s*(detJ1.astype(np.float32)))
            #print(detJ1)
            Floc+=(IntF)

    #2) Add Boundary integral to Floc

    #
    if p==3:
        for i in boundterms:
            # code for boundary term contribution by lower edge

            if enodes[0]==i[1] and enodes[1]==i[2]:#lower edge of tri
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        qx=j[2]
                        #qy=j[3]
                        break
                    else:
                        qx=0
                        #qy=0
                if qx == 0:
                    continue
                print('at lower edge')
                print(enodes)
                #q=Matrix([[qx],[qy]])
                N1=Matrix(N1)
                N1=N1.subs(n,0)

                N=Matrix(N)
                N=N.subs(n,0)
                Nl= np.array(N)

                #Nl =np.array([expr.subs(n, 0) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = (t1**2+t2**2)**0.5

                #l = ((x[0,0]-x[1,0])**2+(x[0,1]-x[1,1])**2)**0.5
                dy= diff(y1,z)
                dx=diff(x1,z)
                q1= Matrix([[dy],[-dx]])*(-qx)/dl
                intF= (N1.T*q1)*(dl)   # it is a matrix with substituted value

                #F1= np.array([[integrate(expr, (z, 0, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,0,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1


                  # Slant edge

            elif enodes[1]==i[1] and enodes[2]==i[2] :
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        qx=j[2]
                        #qy=j[3]
                        break
                    else:
                        qx=0
                        #qy=0
                if qx == 0:
                    continue


                #q=Matrix([[qx],[qy]])
                print(qx)
                N1=Matrix(N1)
                N1=N1.subs(n,1-z)

                N=Matrix(N)
                N=N.subs(n,1-z)
                Nl= np.array(N)

                #Nl =np.array([expr.subs(n, 1-z) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = (t1**2+t2**2)**0.5
                #l = ((x[1,0]-x[2,0])**2+(x[1,1]-x[2,1])**2)**0.5
                dy= diff(y1,z)
                dx=diff(x1,z)
                q1= Matrix([[dy],[-dx]])*(-qx)/dl

                intF= (N1.T*q1)*(dl)
                #intF =np.array([expr.subs(n, 1-z) for expr in intF])
                #F1= np.array([[integrate(expr, (z, 1, 0)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,0,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1


            elif enodes[2]==i[1] and enodes[0]==i[2]:
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        qx=j[2]
                        #qy=j[3]
                        break
                    else:
                        qx=0
                        #qy=0
                if qx == 0:
                    continue
                print('at left edge')
                print(enodes)
                # Left Egde
                #q=Matrix([[qx],[qy]])
                N1=Matrix(N1)
                N1=N1.subs(z,0)

                N=Matrix(N)
                N=N.subs(z,0)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(z, 0) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = (t1**2+t2**2)**0.5
                #l = ((x[2,0]-x[0,0])**2+(x[2,1]-x[0,1])**2)**0.5

                dy= diff(y1,n)
                dx=diff(x1,n)
                q1= Matrix([[dy],[-dx]])*(-qx)/dl
                intF= (N1.T*q1)*(dl)
                #intF =np.array([expr.subs(z, 0) for expr in intF])
                #F1= np.array([[integrate(expr, (n, 0, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((n,0,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1

    else:
        #p=4
        # For Quadrilateral
        for i in boundterms:
            if enodes[3]==i[1] and enodes[0]==i[2]:#lower edge of tri
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        q=j[2]

                        break
                    else:
                        q=0

                if q == 0:
                    continue
                print('calculations for lower edge of quad are going')
                print(enodes)
                # Lower edge
                #q=Matrix([[qx],[qy]])


                N1=Matrix(N1)
                N1=N1.subs(n,-1)

                N=Matrix(N)
                N=N.subs(n,-1)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(n, -1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[0,0]-x[1,0])**2+(x[0,1]-x[1,1])**2)**0.5
                dy= diff(y1,z)
                dx=diff(x1,z)
                q1= Matrix([[dy],[-dx]])*(-q)/dl
                intF= (N1.T*q1)*(dl)
                #intF =np.array([expr.subs(n, -1) for expr in intF])
                #F1= np.array([[integrate(expr, (z, -1, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,-1,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1

             # right edge

            elif enodes[0]==i[1] and enodes[1]==i[2] :
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        qx=j[2]
                        break
                    else:
                        qx=0


                if qx == 0:
                    continue


                print('calculations for right edge of quad are going')
                print(enodes)
                #q=Matrix([[qx],[qy]])
                N1=Matrix(N1)
                N1=N1.subs(z,1)

                N=Matrix(N)
                N=N.subs(z,1)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(z, 1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = (t1**2+t2**2)**0.5
                #l = ((x[1,0]-x[2,0])**2+(x[1,1]-x[2,1])**2)**0.5
                dy= diff(y1,n)
                dx=diff(x1,n)
                q1= Matrix([[dy],[-dx]])*(-qx)/dl

                intF= (N1.T*q1)*(dl)
                #intF =np.array([expr.subs(z, 1) for expr in intF])
                #F1= np.array([[integrate(expr, (n, -1, 1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((n,-1,1))
                F1 =np.array(F1).astype(np.float32)
                Floc+=F1

            # Top egde
            elif enodes[1]==i[1] and enodes[2]==i[2]:
                for j in lpf:
                    if j[1]==i[0]and j[0]==2:
                        qx=j[2]

                        break
                    else:
                        qx=0

                print('it came here')

                if qx == 0:
                    continue
                print('calculations for top edge of quad are going')
                print(enodes)
                #q=Matrix([[qx],[qy]])
                N1=Matrix(N1)
                N1=N1.subs(n,1)

                N=Matrix(N)
                N=N.subs(n,1)
                Nl= np.array(N)
                #Nl =np.array([expr.subs(n, 1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[2,0]-x[3,0])**2+(x[2,1]-x[3,1])**2)**0.5
                dy= diff(y1,z)
                dx=diff(x1,z)
                q1= Matrix([[dy],[-dx]])*(-qx)/dl
                intF= -(N1.T*q1)*(dl)
                #intF =np.array([expr.subs(n, 1) for expr in intF])
                #F1= np.array([[integrate(expr, (z, 1, -1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((z,1,-1))
                F1 =np.array(F1).astype(np.float32)
                print(F1)
                Floc+=F1


                 #left Edge

            elif enodes[2]==i[1] and enodes[3]==i[2]:
                for j in lpf:
                    if j[1]==i[0] and j[0]==2:
                        qx=j[2]
                        #qy=j[3]
                        break
                    else:
                        qx=0

                #print(qy)

                if qx == 0:
                    continue
                print('calculations for lower left of quad are going')
                print(enodes)
                #q=Matrix([[qx],[qy]])
                N1=Matrix(N1)
                N1=N1.subs(z,-1)

                N=Matrix(N)
                N=N.subs(z,-1)
                Nl= np.array(N)
                # Nl =np.array([expr.subs(z, -1) for expr in N])
                x1 =np.dot(x[:,0],Nl[0,:])
                y1 =np.dot(x[:,1],Nl[0,:])
                t1 =diff(x1,z)
                t2= diff(y1,n)
                dl = -(t1**2+t2**2)**0.5
                #l = ((x[2,0]-x[3,0])**2+(x[2,1]-x[3,1])**2)**0.5
                dy= diff(y1,n)
                dx=diff(x1,n)
                q1= Matrix([[dy],[-dx]])*(-qx)/dl

                intF= -(N1.T*q1)*(dl)
                #intF =np.array([expr.subs(z, -1) for expr in intF])
                #F1= np.array([[integrate(expr, (n, 1, -1)) for expr in intF]]).reshape(p,1).astype(np.float32)
                F1 =intF.integrate((n,1,-1))
                F1 =np.array(F1).astype(np.float32)
                print(F1)
                Floc+=F1







    #...
    return Floc

## Question 7:  Problem 2 :

In [ ]:
eleconnect =np.loadtxt('/content/Pr7_4Q_elecon.txt')
nodal_data = np.loadtxt('/content/Pr7_4Q_nodes.txt')

In [ ]:
nnodes = nodal_data[-1, 0].astype(int)
nnodes

81

In [ ]:
nel = eleconnect[-1,0].astype(int)
nel

64

In [ ]:
# LOgic to get boundary nodes
b1=[]
b2=[]
b3=[]
b4=[]
for i in nodal_data:
    if i[1]==0:
        b1.append(i[0]) #left vertical
    elif i[2]==0:
        b2.append(i[0])  # right horizontal
    elif 9.99<(i[1]**2+i[2]**2)**0.5<10.01:   # internal Circle
        b3.append(i[0])
    elif 14.99<(i[1]**2+i[2]**2)**0.5<15.01:  # external Circle
        b4.append(i[0])


In [ ]:
b3.append(1.0) # insert edge nodes
b3.insert(0,4)
b4.append(3.0)
b4.insert(0,2.0)

In [ ]:
# Getting Boundterms
b1 =np.array(b1)
b2 =np.array(b2)
b3 =np.array(b3)
b4 =np.array(b4)
Bound=np.hstack((b1,b2,b3,b4))
boundterms=np.zeros((1,len(Bound)*3))
for i in range(len(Bound)):
    if i<len(b1)-1: # i 0 to 7
        boundterms[0,3*i]=1
        boundterms[0,3*i+1]=Bound[i]
        boundterms[0,3*i+2]=Bound[i+1]
    if len(b1)-1<i<len(b1)+len(b2)-1:  # i =9
        boundterms[0,3*i]=2
        boundterms[0,3*i+1]=Bound[i]
        boundterms[0,3*i+2]=Bound[i+1]
    if len(b1)+len(b2)-1<i<len(b1)+len(b2)+len(b3)-1:
        boundterms[0,3*i]=3
        boundterms[0,3*i+1]=Bound[i]
        boundterms[0,3*i+2]=Bound[i+1]
    if len(b1)+len(b2)+len(b3)-1<i<len(b1)+len(b2)+len(b3)+len(b4)-1:
        boundterms[0,3*i]=4
        boundterms[0,3*i+1]=Bound[i]
        boundterms[0,3*i+2]=Bound[i+1]
boundterms=boundterms.reshape(36,3)
boundterms

array([[ 1.,  3.,  4.],
       [ 1.,  4., 19.],
       [ 1., 19., 20.],
       [ 1., 20., 21.],
       [ 1., 21., 22.],
       [ 1., 22., 23.],
       [ 1., 23., 24.],
       [ 1., 24., 25.],
       [ 0.,  0.,  0.],
       [ 2.,  1.,  2.],
       [ 2.,  2.,  5.],
       [ 2.,  5.,  6.],
       [ 2.,  6.,  7.],
       [ 2.,  7.,  8.],
       [ 2.,  8.,  9.],
       [ 2.,  9., 10.],
       [ 2., 10., 11.],
       [ 0.,  0.,  0.],
       [ 3.,  4., 26.],
       [ 3., 26., 27.],
       [ 3., 27., 28.],
       [ 3., 28., 29.],
       [ 3., 29., 30.],
       [ 3., 30., 31.],
       [ 3., 31., 32.],
       [ 3., 32.,  1.],
       [ 0.,  0.,  0.],
       [ 4.,  2., 12.],
       [ 4., 12., 13.],
       [ 4., 13., 14.],
       [ 4., 14., 15.],
       [ 4., 15., 16.],
       [ 4., 16., 17.],
       [ 4., 17., 18.],
       [ 4., 18.,  3.],
       [ 0.,  0.,  0.]])

In [ ]:
b1

array([ 3.,  4., 19., 20., 21., 22., 23., 24., 25.])

In [ ]:
matching_nodal_data = []

for value in b3:
    row = nodal_data[nodal_data[:, 0] == value]
    if row.size > 0:
        matching_nodal_data.append(row[0])

matching_nodal_data = np.array(matching_nodal_data)

In [ ]:
print(matching_nodal_data)

[[ 4.          0.         10.        ]
 [26.          1.9509042   9.8078528 ]
 [27.          3.82683625  9.23879532]
 [28.          5.55570512  8.31469611]
 [29.          7.07107136  7.0710678 ]
 [30.          8.31470029  5.55570232]
 [31.          9.23879995  3.82683431]
 [32.          9.80785771  1.95090322]
 [ 1.         10.          0.        ]]


In [ ]:
# Getting Hardcoded Fvec
P=2500
Fvec= np.zeros((nnodes*2, 1))

for i in range(len(matching_nodal_data[:,0])-1):
  x1,y1=matching_nodal_data[i,1],matching_nodal_data[i,2]
  x2,y2=matching_nodal_data[i+1,1],matching_nodal_data[i+1,2]
  theta=np.arctan((y2-y1)/x2)
  floc=np.array([[np.cos(theta)],[np.sin(theta)],[np.cos(theta)],[np.sin(theta)]])*P*(((x2-x1)**2+(y2-y1)**2)**0.5)*0.5

  for j in eleconnect:
    if j[5] == matching_nodal_data[i,0] and j[2] == matching_nodal_data[i+1,0]:  # Assuming pressure is at left edge always, which corresponds to the
        enodes=j[2:6]-1
  #print(enodes)
  #print(floc)                                                       # left edge of element, that is joining node(3) and node(0) located at 5th and 2nd position in eleconnect
  Fvec[np.ix_(enodes.astype(int))]+=floc

In [ ]:
boundterms

array([[ 1.,  3.,  4.],
       [ 1.,  4., 19.],
       [ 1., 19., 20.],
       [ 1., 20., 21.],
       [ 1., 21., 22.],
       [ 1., 22., 23.],
       [ 1., 23., 24.],
       [ 1., 24., 25.],
       [ 0.,  0.,  0.],
       [ 2.,  1.,  2.],
       [ 2.,  2.,  5.],
       [ 2.,  5.,  6.],
       [ 2.,  6.,  7.],
       [ 2.,  7.,  8.],
       [ 2.,  8.,  9.],
       [ 2.,  9., 10.],
       [ 2., 10., 11.],
       [ 0.,  0.,  0.],
       [ 3.,  4., 26.],
       [ 3., 26., 27.],
       [ 3., 27., 28.],
       [ 3., 28., 29.],
       [ 3., 29., 30.],
       [ 3., 30., 31.],
       [ 3., 31., 32.],
       [ 3., 32.,  1.],
       [ 0.,  0.,  0.],
       [ 4.,  2., 12.],
       [ 4., 12., 13.],
       [ 4., 13., 14.],
       [ 4., 14., 15.],
       [ 4., 15., 16.],
       [ 4., 16., 17.],
       [ 4., 17., 18.],
       [ 4., 18.,  3.],
       [ 0.,  0.,  0.]])

In [ ]:
# Write logic from boundterm, if edge==1 or edge==2 , for that node only define fdof
f = np.zeros((len(boundterms), 5))
for i in range(len(boundterms[:,0])):
    if boundterms[i,0] == 2:
        f[i,0] = boundterms[i,1]-1           # indices starts from 0
        f[i,1]=0
        f[i,2]=1
        f[i,3]=0
        f[i,4]=0
    elif boundterms[i,0]==1:
        f[i,0] = boundterms[i,1]-1           # indices starts from 0
        f[i,1]=1
        f[i,2]=0
        f[i,3]=0
        f[i,4]=0
        pass
fdof = f[~np.all(f == 0, axis=1)]

In [ ]:
fdof=np.vstack((fdof,np.array([[10,0,1,0,0],[24,1,0,0,0]]))) # adding remaining edge nodes ,manually
fdof

array([[ 2.,  1.,  0.,  0.,  0.],
       [ 3.,  1.,  0.,  0.,  0.],
       [18.,  1.,  0.,  0.,  0.],
       [19.,  1.,  0.,  0.,  0.],
       [20.,  1.,  0.,  0.,  0.],
       [21.,  1.,  0.,  0.,  0.],
       [22.,  1.,  0.,  0.,  0.],
       [23.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  1.,  0.,  0.],
       [ 4.,  0.,  1.,  0.,  0.],
       [ 5.,  0.,  1.,  0.,  0.],
       [ 6.,  0.,  1.,  0.,  0.],
       [ 7.,  0.,  1.,  0.,  0.],
       [ 8.,  0.,  1.,  0.,  0.],
       [ 9.,  0.,  1.,  0.,  0.],
       [10.,  0.,  1.,  0.,  0.],
       [24.,  1.,  0.,  0.,  0.]])

In [ ]:
P=-25000    # N/cm2  =>  25000*10^-2 => 250 Mpa
lpf=np.array([[1,0,0,0],[2,3,P,0]])
#          type , bound no. , Pressure
ngauss=2
E=200e5   # N/cm2
v=0.3
D=np.array([[1-v,v,0],[v,1-v,0],[0,0,(1-2*v)/2]])*(E/((1-2*v)*(1+v)))  # D for Plain Strain

In [ ]:
# Solving By function
U62, F62 =FEM2Dvec(nodal_data, eleconnect,boundterms, fdof, nel, lpf, ngauss,D,four_node_quad,Formload2DPressure)

In [ ]:
U62=U62*5

In [ ]:
def getdefnodes(nnodes,nodal_data,U62):
  U62x=np.zeros(nnodes)
  U62y= np.zeros(nnodes)
  for i in range(len(U62x)):
    U62x[i]=U62[2*i,0]*4              # *4 is scaling factor
    U62y[i]=U62[2*i+1,0]*4
  U62def=np.zeros((len(U62x),2))
  U62def[:,0]=U62x
  U62def[:,1]=U62y
  nodesdef =np.column_stack((nodal_data[:,0], nodal_data[:,1:3]+U62def[:,0:2]))
  return nodesdef


In [ ]:
nodesdef=getdefnodes(nnodes,nodal_data,U62)
nodesdef

NameError: name 'nodal_data' is not defined

In [ ]:
np.savetxt("U7defnodes1.txt", nodesdef)

In [ ]:
Sx=getstress(F62[:,0])

<ipython-input-38-8a0ce5cba44e>:8: RuntimeWarning: invalid value encountered in cast
  Stress[i,:]=np.array([s1,s2,s3,s4])


In [ ]:
np.nanmin(Sx)

-197933.74622529792

In [ ]:
## 1) Get kstiff from code
## Form dirichlet boundary conditions for boundary b1 and b3
## solve for displacements

z,n=symbols('z n')

In [ ]:
F62[0,1].subs(z,1).subs(n,0)

398.383296368294

In [ ]:
# to solve by Hardcode Fvec
dof = np.zeros(nnodes).astype(int)
ndof = 2  # vector field problems
tdof = nnodes*2
count = 0
for i in range(fdof.shape[0]):  # fixeddofs=[0,1,0]
    for j in range(1, ndof + 1):
        if fdof[i, j] == 1:
            dof[count] = fdof[i, 0] * ndof + j - 1
            count += 1

dof = dof[:count]
freedofs = np.setdiff1d(np.arange(0, tdof), dof)

    # Step 4 : Solve KU = F

U = np.zeros([tdof, 1])

if len(dof) == 0:
    print('solving a matrix')
    U = np.linalg.solve(Kstiff, np.zeros(Kstiff.shape[0]))
else:
    U[dof[0]] = fdof[0, -1]
    U[freedofs] = np.linalg.solve(Kstiff[np.ix_(freedofs, freedofs)], Fvec[freedofs] - Kstiff[np.ix_(freedofs, dof)] @ U[dof])